In [25]:
import pandas as pd
import dataframe_image as dfi

## Import data

In [26]:
df_nodes_genes = pd.read_csv('../processed_data/nodes_genes.csv')

# Edges: Protein-Gene Connection
File from: https://www.ensembl.org/biomart/martview/

Ensembl Genes 112 > Human Genes (GRCh38.p14)

Columns: Gene stable ID, Protein stable ID

In [27]:
df_edges_protein_gene = pd.read_csv('../import_data/ENSEMBLE/biomart_gene_protein.txt',  sep='\t')
df_edges_protein_gene.rename(columns={'Gene stable ID': 'Gene ID', 'Protein stable ID': 'Protein ID'}, inplace=True)

# filter for genes in nodes
df_edges_protein_gene = df_edges_protein_gene[df_edges_protein_gene['Gene ID'].isin(df_nodes_genes['Gene ID'])]

# drop genes without protein
df_edges_protein_gene.dropna(subset=['Protein ID'], inplace=True)

df_edges_protein_gene

,Gene ID,Protein ID
5,ENSG00000198888,ENSP00000354687
9,ENSG00000198763,ENSP00000355046
15,ENSG00000198804,ENSP00000354499
18,ENSG00000198712,ENSP00000354876
20,ENSG00000228253,ENSP00000355265
...,...,...
185320,ENSG00000143520,ENSP00000373370
185321,ENSG00000143507,ENSP00000355866
185322,ENSG00000143507,ENSP00000483812
185323,ENSG00000143507,ENSP00000482935


# Edges: Protein-Protein Interaction
STRING Data as Interaction

In [28]:
filepath = '../import_data/STRING/protein.links.full.v12.0.txt'
df_edges_protein = pd.read_csv(filepath, sep=' ', usecols=[0, 1])

print('Length of STRING Links Dataset: ', len(df_edges_protein))

df_edges_protein.rename(columns={'protein1': 'left Protein ID', 'protein2': 'right Protein ID'}, inplace=True)

df_edges_protein['left Protein ID'] = df_edges_protein['left Protein ID'].apply(lambda x: x.split('.')[1])
df_edges_protein['right Protein ID'] = df_edges_protein['right Protein ID'].apply(lambda x: x.split('.')[1])


df_edges_protein

Length of STRING Links Dataset:  13715404


,left Protein ID,right Protein ID
0,ENSP00000000233,ENSP00000356607
1,ENSP00000000233,ENSP00000427567
2,ENSP00000000233,ENSP00000253413
3,ENSP00000000233,ENSP00000493357
4,ENSP00000000233,ENSP00000324127
...,...,...
13715399,ENSP00000501317,ENSP00000475489
13715400,ENSP00000501317,ENSP00000370447
13715401,ENSP00000501317,ENSP00000312272
13715402,ENSP00000501317,ENSP00000402092


### Filter for Proteins from Nodes
Leads to only direct connections between proteins that are connected to genes

→ Gene - Protein - Protein - Gene


But we also want to include proteins that are not connected to a gene

→ Gene - Protein - $Protein$ - Protein - Gene


In [29]:
"""df_edges_protein = df_edges_protein[df_edges_protein['protein1'].isin(df_nodes_protein)]
df_edges_protein = df_edges_protein[df_edges_protein['protein2'].isin(df_nodes_protein)]

df_edges_protein"""

"df_edges_protein = df_edges_protein[df_edges_protein['protein1'].isin(df_nodes_protein)]\ndf_edges_protein = df_edges_protein[df_edges_protein['protein2'].isin(df_nodes_protein)]\n\ndf_edges_protein"

# Nodes: Protein



In [30]:
# Proteins that are connected to genes
df_nodes_protein1 = df_edges_protein_gene.copy()
df_nodes_protein1 = df_nodes_protein1['Protein ID']

df_nodes_protein1 = df_nodes_protein1.drop_duplicates()

df_nodes_protein1

5         ENSP00000354687
9         ENSP00000355046
15        ENSP00000354499
18        ENSP00000354876
20        ENSP00000355265
               ...       
185320    ENSP00000373370
185321    ENSP00000355866
185322    ENSP00000483812
185323    ENSP00000482935
185324    ENSP00000480008
Name: Protein ID, Length: 101731, dtype: object

In [31]:
# proteins that are connected to other proteins
df_nodes_protein2 = pd.concat([df_edges_protein['left Protein ID'], df_edges_protein['right Protein ID']], ignore_index=True)
df_nodes_protein2 = df_nodes_protein2.drop_duplicates()
df_nodes_protein2.rename('Protein ID', inplace=True)

df_nodes_protein2

0           ENSP00000000233
1301        ENSP00000000412
1667        ENSP00000001008
3113        ENSP00000001146
4274        ENSP00000002125
                 ...       
13711347    ENSP00000501254
13712993    ENSP00000501259
13713530    ENSP00000501265
13714311    ENSP00000501277
13715044    ENSP00000501317
Name: Protein ID, Length: 19622, dtype: object

In [32]:
# combine both
df_nodes_protein = pd.concat([df_nodes_protein1, df_nodes_protein2], ignore_index=True)
df_nodes_protein = df_nodes_protein.drop_duplicates()

df_nodes_protein

0         ENSP00000354687
1         ENSP00000355046
2         ENSP00000354499
3         ENSP00000354876
4         ENSP00000355265
               ...       
121335    ENSP00000501085
121336    ENSP00000501088
121337    ENSP00000501092
121339    ENSP00000501111
121352    ENSP00000501317
Name: Protein ID, Length: 104235, dtype: object

# Export Data

In [33]:
df_nodes_protein.to_csv('../processed_data/nodes_protein.csv', index=False)
df_edges_protein.to_csv('../processed_data/edges_protein.csv', index=False)
df_edges_protein_gene.to_csv('../processed_data/edges_protein_gene.csv', index=False)

In [34]:
print(f"There are {len(df_nodes_protein)} Protein Nodes")
print(f"There are {len(df_edges_protein)} Protein-Protein Edges")
print(f"There are {len(df_edges_protein_gene)} Protein-Gene Edges")

There are 104235 Protein Nodes
There are 13715404 Protein-Protein Edges
There are 101731 Protein-Gene Edges


In [35]:
df_edges_protein_plot = df_edges_protein.reset_index(drop=True)
df_edges_protein_gene_plot = df_edges_protein_gene.reset_index(drop=True)
df_nodes_protein_plot = df_nodes_protein.reset_index()

df_nodes_protein_plot
# lib cant plot single columns

,index,Protein ID
0,0,ENSP00000354687
1,1,ENSP00000355046
2,2,ENSP00000354499
3,3,ENSP00000354876
4,4,ENSP00000355265
...,...,...
104230,121335,ENSP00000501085
104231,121336,ENSP00000501088
104232,121337,ENSP00000501092
104233,121339,ENSP00000501111


In [36]:
dfi.export(df_edges_protein_plot.head(5), '../tex/figures/03_02_protein_edges.png')
dfi.export(df_edges_protein_gene_plot.head(5), '../tex/figures/03_02_gene_protein_edges.png')
dfi.export(df_nodes_protein_plot.head(5), '../tex/figures/03_02_protein_nodes.png')